# Exercise  

We have a layout file giving us the information on how we must interprete each file.

Files have their generation date in the file name with the format PYYMMDD.

We must create a class that takes each file, translates it and returns to us a DataFrame.

The dataframe columns must be:  
ID as portfolio_ID   
target ID as Amount   
operation date as trade date  
valuation date as settle date  
operation descritpion as desc (Internal Transaction)  
A column named estimation with values (E)  
We must include a column with the currency (MXN)  
And a column with a transaction_ID by concatenating three fields: portfolio_ID, Settle_date, operation descrition   

The final file must have all the processed transactions and have the filename generated with the next business day 
"cash_transactions_DATE.csv"

In [51]:
%%writefile layout_config.py
LAYOUT_MAPPING = {
    "ID":(1,3),
    "Price":(4,15),
    "OPERATION DATE":(16,23),
    "VALUATION DATE":(24,31),
    "TARGET VALUE":(32,46),
    "AMOUNT 1":(47,63),
    "AMOUNT 2":(64,80),
    "AMOUNT 3":(81,97),
    "AMOUNT 4":(98,114),
    "AMOUNT 5":(115,131),
    "AMOUNT 6":(132,148),
    "AMOUNT 7":(149,165)
}

COLUMNS_MAPPING = {
    "ID":"PORTFOLIO_ID",
    "OPERATION DATE":"TRADE_DATE",
    "VALUATION DATE":"SETTLE_DATE",
    "TARGET VALUE":"AMOUNT"
}

Overwriting layout_config.py


In [16]:
%reload_ext autoreload
%autoreload 2

In [80]:
#%%writefile processing.py
import pandas as pd
from layout_config import LAYOUT_MAPPING, COLUMNS_MAPPING

class ProcessFile():
    def __init__(self, filename):
        self._filename = filename
        
        self.clean_data()
    
    def clean_data(self) -> None:
        """
        The clean_data function reads the file, extracts the layout and creates a dataframe with 
        the transaction information. The function returns a dataframe with all transactions.
        
        Returns:
            The transaction_df dataframe
        
        """
    
        def _read_file() -> None:
            with open(self._filename, "r") as Pfile:
                self.string = Pfile.read()  
            
            return  
        
        def _get_layout() -> None:
            get_interval = lambda target_string, interval: target_string[interval[0] -1:interval[1]] 
            self.layout = {key:get_interval(self.string, interval) for key, interval in LAYOUT_MAPPING.items()}
            
            return
    
        def _process_layout() -> None:
            self.layout_df = pd.DataFrame.from_dict(self.layout, orient="index").T
            self.layout_df[["OPERATION DATE", "VALUATION DATE"]] = self.layout_df[["OPERATION DATE", "VALUATION DATE"]].apply(pd.to_datetime)
            self.layout_df.rename(columns=COLUMNS_MAPPING, inplace=True)
            return
        
        def _set_final_dataframe() -> None:
            self.transaction_df = self.layout_df[['PORTFOLIO_ID', 'TRADE_DATE', 'SETTLE_DATE', 'AMOUNT']].copy()
            self.transaction_df["DESC"] = "Internal Transacion"
            self.transaction_df["ESTIMATED"] = "E"
            self.transaction_df["CURRENCY"] = "MXN"
            
            self.transaction_df["TRANSACTION_ID"] = (self.transaction_df["PORTFOLIO_ID"] 
                                                     + '_' 
                                                     + self.transaction_df["SETTLE_DATE"].dt.strftime("%Y%m%d")
                                                     + '_'
                                                     + self.transaction_df["DESC"].str.replace(' ', "_").str.upper()
                                                     + '_'
                                                     + self.transaction_df["ESTIMATED"]
                                                     )
            
            self.transaction_df['TRADE_DATE'] = self.transaction_df['TRADE_DATE'].dt.strftime("%d%m%Y")
            self.transaction_df['SETTLE_DATE'] = self.transaction_df['SETTLE_DATE'].dt.strftime("%d%m%Y")

            return
        
        # main function(clean_data)
        _read_file()
        _get_layout()
        _process_layout()
        _set_final_dataframe()
        
        return
    

In [81]:
test_filename = r"C:\Users\2000076134\Dev\OneDrive_1_9-30-2022\P220602.955"
transaction_data = ProcessFile(test_filename)
df = transaction_data.transaction_df
df

,PORTFOLIO_ID,TRADE_DATE,SETTLE_DATE,AMOUNT,DESC,ESTIMATED,CURRENCY,TRANSACTION_ID
0,559,02062022,03062022,57197.58,Internal Transacion,E,MXN,559_20220603_INTERNAL_TRANSACION_E


In [82]:
import os

In [108]:
file_path = r"C:\Users\2000076134\Dev\OneDrive_1_9-30-2022"
file_list = os.listdir(file_path)
file_list = list(filter(lambda file: re.match('P\d{6}.\d{2}', file) ,file_list))

In [117]:
source_df = pd.DataFrame()

for file in file_list:
    file = os.path.join(file_path, file)
    transaction_data = ProcessFile(file)
    
    source_df = pd.concat([source_df, transaction_data.transaction_df])
    

In [118]:
source_df

,PORTFOLIO_ID,TRADE_DATE,SETTLE_DATE,AMOUNT,DESC,ESTIMATED,CURRENCY,TRANSACTION_ID
0,559,02062022,03062022,57197.58,Internal Transacion,E,MXN,559_20220603_INTERNAL_TRANSACION_E
0,559,03062022,06062022,171645.18,Internal Transacion,E,MXN,559_20220606_INTERNAL_TRANSACION_E
0,559,06062022,07062022,57014.87,Internal Transacion,E,MXN,559_20220607_INTERNAL_TRANSACION_E
0,559,07062022,08062022,56797.52,Internal Transacion,E,MXN,559_20220608_INTERNAL_TRANSACION_E
0,559,08062022,09062022,56957.92,Internal Transacion,E,MXN,559_20220609_INTERNAL_TRANSACION_E
0,559,09062022,10062022,56987.03,Internal Transacion,E,MXN,559_20220610_INTERNAL_TRANSACION_E
0,559,10062022,13062022,170306.84,Internal Transacion,E,MXN,559_20220613_INTERNAL_TRANSACION_E
0,559,13062022,14062022,56539.68,Internal Transacion,E,MXN,559_20220614_INTERNAL_TRANSACION_E
0,559,14062022,15062022,56199.54,Internal Transacion,E,MXN,559_20220615_INTERNAL_TRANSACION_E


In [86]:
s = 'P220602.955'

In [92]:
s.split(".")[1].isdigit()

True

In [96]:
len(s.split(".")[1]) == 3

True

In [97]:
[file for file in file_list
 if file.startswith("P") and s.split(".")[1].isdigit()]

['P220602.955',
 'P220603.955',
 'P220606.955',
 'P220607.955',
 'P220608.955',
 'P220609.955',
 'P220610.955',
 'P220613.955',
 'P220614.955']

In [98]:
import re

In [103]:
[file for file in file_list 
 if re.match('P\d{6}.\d{2}', file)]

['P220602.955',
 'P220603.955',
 'P220606.955',
 'P220607.955',
 'P220608.955',
 'P220609.955',
 'P220610.955',
 'P220613.955',
 'P220614.955']

In [105]:
list(filter(lambda file: re.match('P\d{6}.\d{2}', file) ,file_list))

['P220602.955',
 'P220603.955',
 'P220606.955',
 'P220607.955',
 'P220608.955',
 'P220609.955',
 'P220610.955',
 'P220613.955',
 'P220614.955']